# NDWI composites

For use in stratifying WOfS validation


### Load packages

In [1]:
%matplotlib inline

import geopandas as gpd
import datacube
from datacube.utils import geometry
# from datacube.helpers import write_geotiff
from datacube.utils.cog import write_cog
import warnings
warnings.filterwarnings("ignore")

import os
import xarray as xr
import numpy as np


import sys
sys.path.append('../Scripts')
from deafrica_datahandling import load_ard
from deafrica_dask import create_local_dask_cluster
from deafrica_spatialtools import xr_rasterize
from deafrica_classificationtools import HiddenPrints
from deafrica_plotting import map_shapefile
from deafrica_bandindices import calculate_indices

### Set up a dask cluster

In [2]:
create_local_dask_cluster(aws_unsigned=False)

Client Scheduler: tcp://127.0.0.1:46723 Dashboard: /user/fangfy/proxy/45915/status,Cluster Workers: 1 Cores: 15 Memory: 104.37 GB


### Connect to the datacube

In [3]:
dc = datacube.Datacube(app='Geomedian_composites')

## Analysis Parameters

In [4]:
area = 'Central'

In [5]:
time_range = ('2013','2019')

vector_file = '../../shapes/ls8_tiles/tiles.shp'
attribute_col = 'id'
aez_region = f'../../shapes/simplified_AEZs/{area}.shp'

products = ['usgs_ls8c_level2_2']
measurements = ['green', 'nir']
resolution = (-30, 30)
output_crs = 'EPSG:6933'
align = (15,15)
dask_chunks= {'x':2500,'y':2500, 'time':1}

export_prefix=f"NDWI_composite/{area.lower()}_NDWI_tile-"
if not os.path.exists(os.path.dirname(export_prefix)): os.system('mkdir -p '+os.path.dirname(export_prefix))

### Set up a query

In [6]:
# Create a reusable query
query = {'time': time_range,
         'measurements': measurements,
         'resolution': resolution,
         'output_crs': output_crs,
         'align': align,
         'dask_chunks':dask_chunks
         }

query

{'time': ('2013', '2019'),
 'measurements': ['green', 'nir'],
 'resolution': (-30, 30),
 'output_crs': 'EPSG:6933',
 'align': (15, 15),
 'dask_chunks': {'x': 2500, 'y': 2500, 'time': 1}}

### Open and clip shapefiles

Open `tiles.shp` and clip it to the boundary shapefile (in this example the `aez_region`)


In [7]:
#read shapefile
gdf = gpd.read_file(vector_file)

#open shapefile
aez=gpd.read_file(aez_region)

# clip points to region
gdf = gpd.overlay(gdf, aez, how='intersection')

### Add a unique identifier

and plot the tiles using `map_shapefile`, you can hover your mouse over the tiles to see the ID 

In [8]:
# add an ID column
gdf[attribute_col]=range(0, len(gdf))

#print gdf
# map_shapefile(gdf, attribute_col)

In [9]:
len(gdf)

294

### Generate a NDWI composite


In [ ]:
%%time

# Loop through polygons in geodataframe and extract satellite data
for index, row in gdf.iloc[gdf['id'].loc[0:]].iterrows():
    
    output_name = export_prefix+str(row[attribute_col]) + '.tif'
    if os.path.exists(output_name):
        # redo tiles with missing data
        data = xr.open_rasterio(output_name)
        bad_frac = np.isnan(data).mean().values
        if bad_frac < 0.0001: continue
        t = os.system('rm '+output_name)
    
    print("Working on tile: "+str(gdf['id'][index]))
    
    # Get the geometry
    geom = geometry.Geometry(row.geometry.__geo_interface__,
                             geometry.CRS(f'EPSG:{gdf.crs.to_epsg()}'))
    
    # Update dc query with geometry      
    query.update({'geopolygon': geom}) 
    
    with HiddenPrints():
        try:
            ds = load_ard(dc=dc, 
                          products=products,                 
                          group_by='solar_day',
                          **query)
        except: 
            continue
    
    # Generate a polygon mask to keep only data within the polygon
#     with HiddenPrints():
#         mask = xr_rasterize(gdf.iloc[[index]], ds)
    
#     # Mask dataset to set pixels outside the polygon to `NaN`
#     ds = ds.where(mask)
    
    with HiddenPrints():
        ds = calculate_indices(ds, 'NDWI',collection='c2', drop=True)
    
    ndwi = ds.NDWI.median('time') 
    
    print("     Writing to file...")    
    write_cog(ndwi.compute(), output_name)
    

Working on tile: 21
Working on tile: 32
     Writing to file...
Working on tile: 33
     Writing to file...
Working on tile: 34
     Writing to file...


---
Code to get geojson from datacube UI 

In [11]:
# from datacube import Datacube
# from odc.ui import DcViewer
# dc = Datacube()
# ui = DcViewer(dc, "2018", products=['ga_ls8c_gm_2_annual'])
# ui

# x = ui._gui.map.layers[1].data

# from shapely.geometry import shape
# geom = [shape(i) for i in x['features']]
# gdf = gpd.GeoDataFrame({'geometry':geom}, crs='EPSG:4326')
# gdf.to_file('data/tiles.shp')